# Imaging Performance Comparison (Quiet Sun)
This notebook loops over different noise levels and calibration errors to generate, corrupt, image, and compare solar flare datasets.

In [ ]:
import fasr_solar_simul as fss
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1
from scipy.constants import c
import os
from casatasks import tclean, applycal, clearcal
from datetime import datetime
from pathlib import Path
from tqdm import tqdm

# Speed of light in m/s
C_LIGHT = c
import matplotlib
%matplotlib inline

In [ ]:
from importlib import reload

reload(fss)
target = 'Quiet_Sun'

## Setup a Log Spiral Configuration

In [ ]:
# %matplotlib notebook
%matplotlib inline
from importlib import reload

reload(fss)
# Define log-spiral parameters
array_config = {"n_arms": 3, "antennas_per_arm": 56, "alpha": 1.0, "gamma": 0.55, "r0": 1.5, "r_max": 1800, "n_turn": 2,
                "latitude": 35}

# for gam in np.linspace(0.3, 1, 8):
# array_config = {"n_arms": 3, "antennas_per_arm": 56, "alpha": 1.0, "gamma": gam, "r0": 1.5, "r_max": 1800, "n_turn": 2,
#                 "latitude": 35}
positions_logspiral = fss.generate_log_spiral_antenna_positions(**array_config)
figname = (
    f"fig-fasr_Log_Spiral-{len(positions_logspiral)}-"
    f"n_arms={array_config['n_arms']}-antennas_per_arm={array_config['antennas_per_arm']}-"
    f"alpha={array_config['alpha']:.2f}-gamma={array_config['gamma']:.2f}-"
    f"r0={array_config['r0']:.1f}-r_max={array_config['r_max']:.0f}-n_turn={array_config['n_turn']:.1f}.jpg")
print(f'fig saved to {figname}')
figsubfolder = figname.rstrip('.jpg')
formatted_params = [
    rf"$n_{{\rm arms}}={array_config['n_arms']}$",
    rf"$\mbox{{antennas per arm}}={array_config['antennas_per_arm']}$",
    rf"$\alpha={array_config['alpha']:.2f}$",
    rf"$\gamma={array_config['gamma']:.2f}$",
    rf"$r_0={array_config['r0']:.1f}\,\mbox{{m}}$",
    rf"$r_{{\rm max}}={array_config['r_max']:.0f}\,\mbox{{m}}$",
    rf"$n_{{\rm turn}}={array_config['n_turn']:.1f}$"
]

# Create a full string that also includes the spiral equation.
array_config_str = (
    rf"$n_{{arms}}={array_config['n_arms']}, "
    rf"n_{{perarm}}={array_config['antennas_per_arm']}, "
    rf"\alpha={array_config['alpha']:.2f}, \gamma={array_config['gamma']:.2f}, "
    rf"r_0={array_config['r0']:.1f}\,m, r_{{max}}={array_config['r_max']:.0f}\,m, "
    rf"n_{{turn}}={array_config['n_turn']:.1f}$"
)
fss.plot_all_panels(positions_logspiral, "Log Spiral", frequency=2, figname=figname,
                    array_config_str=array_config_str)


## Save the cfg file

In [ ]:
config_file = f"fasr_Log_Spiral-{len(positions_logspiral)}.cfg"
os.system('rm -rf ' + config_file)
fss.write_casa_antenna_list(config_file, positions_logspiral)

# Imaging Performance Comparison
looping over different noise levels and calibration errors to generate, corrupt, image, and compare

## set up the simulation

In [ ]:
# List of all available frequencies in GHz
freqlistALL = ['2GHz', '4GHz', '8GHz', '10GHz', '20GHz']

# List of configurations
project = 'FASR'
config_files = ['fasr_Log_Spiral-168.cfg', 'fasr-a-spiral-168-opt.cfg']

## Antenna temperature noise levels
noise_levels = ['0.005MK', '0.01MK'][:1]

## Define the fractional gain errors tuples: (phase and amplitude)
cal_errors = [(0.2, 0.2), (0.1, 0.1), (0.05, 0.05), (0.02, 0.02)]

# Select frequencies for processing
freq_list = freqlistALL[:-1]

# Reference timestamp for the observation series
reftime_obj = datetime(2020, 11, 26, 20, 0, 0)

# Flags to control overwriting existing data products
overwrite_ms   = False  # Overwrite measurement set?
overwrite_im   = False  # Overwrite image products?
overwrite_plot = False  # Overwrite existing plots?

# Deconvolution algorithm choice
deconvolver = 'hogbom'
# deconvolver = 'multiscale' ## this will be much slower than hogbom

# Imaging parameters (in seconds)
integration_time = 1  # Time per integration
duration         = 1  # Total imaging duration

## predict obs and image

In [ ]:
%matplotlib inline
reload(fss)

reftime = reftime_obj.strftime('%Y/%m/%d/%H:%M:%S')

figdir = os.path.join(project, figsubfolder)
if not os.path.exists(figdir):
    os.makedirs(figdir)

for config_file in config_files:
    cfg_suffix = os.path.basename(config_file.rstrip(".cfg"))
    for freqstr in freq_list:
        solar_model = f'Quiet_Sun/solar_disk_model_20201126.{freqstr}.fits'
        for noise in noise_levels:
            for phaerr, amperr in cal_errors:
                gaintable = [f'caltb_FASR_corrupt_{np.int_(phaerr * 100)}pct.amp',
                             f'caltb_FASR_corrupt_{np.int_(amperr * 100)}pct.ph']
                msname = fss.make_msname(project, target, freqstr, reftime_obj, duration, integration_time, config_file, noise)
                imname = fss.make_imname(msname, deconvolver, phaerr, amperr)
                msfile = f'{msname}.ms'
                if os.path.exists(msfile):
                    if overwrite_ms:
                        os.system('rm -rf ' + msfile)
                    else:
                        pass

                if not os.path.exists(msfile):
                    fss.generate_ms(config_file, solar_model, reftime, freqstr, integration_time=integration_time,
                                    duration=duration,
                                    msname=msfile, noise=noise)

                if os.path.exists(f'{imname}.image'):
                    if overwrite_im:
                        junk = ['.image', '.model', '.mask', '.pb', '.psf', '.residual', '.sumwt']
                        for j in junk:
                            os.system(f'rm -rf {imname}{j}')

                if not os.path.exists(f'{imname}.image'):
                    if not os.path.exists(gaintable[0]):
                        gaintable = fss.generate_caltb(msfile, caltype=['ph', 'amp'], calerr=[phaerr, amperr])
                    clearcal(vis=msfile)
                    applycal(vis=msfile, gaintable=gaintable, applymode='calonly', calwt=False)
                    tclean(vis=msfile, imagename=imname,
                           datacolumn='corrected',
                           field='', spw='', specmode='mfs', deconvolver=deconvolver,
                           imsize=2048, cell=['1.2arcsec'],
                           weighting='briggs', robust=-0.5,
                           niter=5000,
                           interactive=False)
                    junk = ['.model', '.mask', '.pb', '.psf', '.residual', '.sumwt']
                    for j in junk:
                        os.system(f'rm -rf {imname}{j}')

                figname = os.path.join(figdir, f'fig-{os.path.basename(imname)}.jpg')
                if os.path.exists(figname):
                    if overwrite_plot:
                        os.system(f'rm -rf {figname}')

                if not os.path.exists(figname):
                    reload(fss)
                    ## cleaned image
                    image1 = imname + '.image'
                    ## model image
                    image2 = os.path.join(project, os.path.basename(solar_model.replace('.fits', '.im')))
                    ## meta information of the images
                    image_meta = {'title': [f'Image ({deconvolver})', 'Model (convolved)'],
                                  'freq': freqstr.lstrip("0"),
                                  'array_config': config_file,
                                  'noise': f'{1e6 * float(noise.rstrip("MK")):.0f}K',
                                  'cal_error': f'{np.int_(phaerr * 100)}% Pha & {np.int_(amperr * 100)}% Amp',
                                  }

                    fig, axs = fss.plot_two_casa_images_with_convolution(image1, image2,
                                                                         crop_fraction=(0.0, 1.0),
                                                                         figsize=(15, 4),
                                                                         image_meta=image_meta,
                                                                         cmap='inferno',
                                                                         vmax=95, vmin=-10,
                                                                         compare_two=False,
                                                                         contour_levels=[0.025, 0.05, 0.1, 0.2, 0.4,
                                                                                         0.8],
                                                                         conv_tag=f'.{cfg_suffix}',
                                                                         overwrite_conv=False)

                    fig.savefig(figname, dpi=300)

## plot imaging results and compare

In [ ]:
from importlib import reload
%matplotlib inline
reload(fss)
figdir = os.path.join(project, figsubfolder)
config_file1 = config_files[0]
cfg_suffix1 = os.path.basename(config_file1.rstrip(".cfg"))
config_file2 = config_files[1]
cfg_suffix2 = os.path.basename(config_file2.rstrip(".cfg"))

for freqstr in freq_list:
    solar_model = f'Quiet_Sun/solar_disk_model_20201126.{freqstr}.fits'
    image1_model = os.path.join(project, os.path.basename(solar_model.replace('.fits', f'.{cfg_suffix1}.im.convolved')))
    image2_model = os.path.join(project, os.path.basename(solar_model.replace('.fits', f'.{cfg_suffix2}.im.convolved')))

    for noise in noise_levels:
        msname1 = fss.make_msname(project,target, freqstr, reftime_obj, duration, integration_time, config_file1, noise)
        msname2 = fss.make_msname(project,target, freqstr, reftime_obj, duration, integration_time, config_file2, noise)
        for phaerr, amperr in cal_errors:
            figname = os.path.join(figdir,
                                   f'fig-{target}-{cfg_suffix1}-vs-{cfg_suffix2}_noise{noise}_phaerr{np.int_(phaerr * 100)}pct_amperr{np.int_(amperr * 100)}pct-{deconvolver}-{freqstr}-img-fidelity.jpg')
            if os.path.exists(figname):
                if overwrite_plot:
                    os.system(f'rm -rf {figname}')

            figname_blowup = os.path.join(figdir,
                                    f'fig-{target}-{cfg_suffix1}-vs-{cfg_suffix2}_noise{noise}_phaerr{np.int_(phaerr * 100)}pct_amperr{np.int_(amperr * 100)}pct-{deconvolver}-{freqstr}-img-fidelity_blowup.jpg')

            if os.path.exists(figname_blowup):
                if overwrite_plot:
                    os.system(f'rm -rf {figname_blowup}')

            figname_blowup2 = os.path.join(figdir,
                                           f'fig-{target}-{cfg_suffix1}-vs-{cfg_suffix2}_noise{noise}_phaerr{np.int_(phaerr * 100)}pct_amperr{np.int_(amperr * 100)}pct-{deconvolver}-{freqstr}-img-fidelity_limbAR_blowup.jpg')

            if os.path.exists(figname_blowup2):
                if overwrite_plot:
                    os.system(f'rm -rf {figname_blowup2}')

            imname1 = fss.make_imname(msname1, deconvolver, phaerr, amperr)
            image1 = imname1 + '.image'
            title1 = f'{freqstr} {cfg_suffix1} {deconvolver}'

            imname2 = fss.make_imname(msname2, deconvolver, phaerr, amperr)
            image2 = imname2 + '.image'
            title2 = f'{freqstr} {cfg_suffix2} {deconvolver}'
            image_meta = {'title': [f'Image ({deconvolver})', f'Image ({deconvolver})'],
                          'freq': freqstr.lstrip("0"),
                          'array_config': config_files,
                          'noise': f'{1e6 * float(noise.rstrip("MK")):.0f}K',
                          'cal_error': f'{np.int_(phaerr * 100)}% Pha & {np.int_(amperr * 100)}% Amp',
                          }
            if not os.path.exists(figname):
                fig, axs = fss.plot_two_casa_images(image1, image2,
                                                    crop_fraction=(0.0, 1.0),
                                                    figsize=(10, 4),
                                                    image_meta=image_meta,
                                                    cmap='inferno',
                                                    cmap_model='turbo',
                                                    vmax=99, vmin=-20,
                                                    vmax_percentile=99.999, vmin_percentile=1,
                                                    uni_vmaxmin=True,
                                                    image1_model_filename=image1_model,
                                                    image2_model_filename=image2_model,
                                                    contour_levels=[0.025, 0.05, 0.1, 0.2, 0.4, 0.8])
                fig.savefig(figname, dpi=300)
            if not os.path.exists(figname_blowup):
                fig, axs = fss.plot_two_casa_images(image1, image2,
                                                    crop_fraction=((0.17, 0.37), (0.25, 0.45)),
                                                    figsize=(10, 4),
                                                    image_meta=image_meta,
                                                    cmap='inferno',
                                                    cmap_model='turbo',
                                                    vmax=99, vmin=-20,
                                                    vmax_percentile=99.999, vmin_percentile=1,
                                                    uni_vmaxmin=True,
                                                    image1_model_filename=image1_model,
                                                    image2_model_filename=image2_model,
                                                    contour_levels=[0.025, 0.05, 0.1, 0.2, 0.4, 0.8])

                fig.savefig(figname_blowup, dpi=300)
            if not os.path.exists(figname_blowup2):
                fig, axs = fss.plot_two_casa_images(image1, image2,
                                                    crop_fraction=((0.05, 0.25), (0.7-0.075, 0.9-0.075)),
                                                    figsize=(10, 4),
                                                    image_meta=image_meta,
                                                    cmap='inferno',
                                                    cmap_model='turbo',
                                                    vmax=99, vmin=-20,
                                                    vmax_percentile=99.999, vmin_percentile=1,
                                                    uni_vmaxmin=True,
                                                    image1_model_filename=image1_model,
                                                    image2_model_filename=image2_model,
                                                    contour_levels=[0.025, 0.05, 0.1, 0.2, 0.4, 0.8])
                fig.savefig(figname_blowup2, dpi=300)